In [1]:
# This information helps with debugging and getting support :)
import sys, platform
import pandas as pd
import bifacial_radiance as br
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
Pandas version  1.5.3
bifacial_radiance version  0.4.3.dev341+g93e0ec5.d20240830


# 21 - Weather to Module Performance
## Modeling Performance, an End to End Simulation

This tutorial shows how to use the new function on bifacial_radiance calculatePerformanceModule performance, as well as how to find CEC Module parameters.


In [2]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_21')

if not os.path.exists(testfolder): os.mkdir(testfolder)

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21


In [3]:
import bifacial_radiance
import numpy as np
import pandas as pd
import pvlib 

bifacial_radiance.__version__

'0.4.3.dev341+g93e0ec5.d20240830'

## Geting a CEC Module to pass into demo.makeModule

In [4]:
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
db = pd.read_csv(url, index_col=0) # Reading this might take 1 min or so, the database is big.

Find the module that you want. In this case we know it's a SunPower of model SPR-E19-310-COM. 

Make sure you select only 1 module from the database -- sometimes there are similar names.

In [5]:
modfilter2 = db.index.str.startswith('SunPower') & db.index.str.endswith('SPR-E19-310-COM')
print(modfilter2)
CECMod = db[modfilter2]
print(len(CECMod), " modules selected. Name of 1st entry: ", CECMod.index[0])

[False False False ... False False False]
1  modules selected. Name of 1st entry:  SunPower SPR-E19-310-COM


In [6]:
# Selecting only two times as examples
starttime = '01_13_11';  endtime = '01_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
demo.setGround(0.2)

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


The CEC data should be passed into the ModuleObj, either at time of creation, or sometime before it is passed into makeScene.

In [7]:
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



The same data could instead be passed after the ModuleObj's definition, or at time of performance analysis:

In [8]:
mymodule.addCEC(CECMod)

In [9]:
# Let's make a second module, and set it to the default Prism Solar module type
mymodule2 = demo.makeModule(name='test', x=1, y=2, bifi=0.8, CECMod=None) 


Module Name: test
Module test updated in module.json
Pre-existing .rad file objects\test.rad will be overwritten



We're going to set up two scenes, each with a different module type!

In [10]:
sceneDict = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':5, 'nRows': 2}
sceneDict2 = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':2.5, 'nMods':2, 'nRows': 1, 'originx': -15}
trackerdict = demo.set1axis(metdata = metdata, cumulativesky = False)
trackerdict = demo.gendaylit1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/

Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


Make a second scene with the other module type

In [11]:
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule2, sceneDict=sceneDict2, append=True)
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(sensorsy=3, append=False)
trackerdict = demo.analysis1axis(sensorsy=3, sceneNum=1)


Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2021-01-13_1100.oct
Created 1axis_2021-01-13_1200.oct
Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module3_Front


c:\users\cdeline\documents\python scripts\bifacial_radiance\bifacial_radiance\main.py:2842: UserWarning: Append=False. Over-writing any existing `AnalysisObj` in trackerdict.
  warnings.warn('Append=False. Over-writing any existing `AnalysisObj` in trackerdict.')


Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-01-13_1100_Scene0_Row1_Module3.csv
Index: 2021-01-13_1100. Wm2Front: 254.58006666666665. Wm2Back: 39.917336666666664
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-01-13_1200_Scene0_Row1_Module3.csv
Index: 2021-01-13_1200. Wm2Front: 253.3529. Wm2Back: 38.14312
Linescan in process: 1axis_2021-01-13_1100_Scene1_Row1_Module1_Front
Linescan in process: 1axis_2021-01-13_1100_Scene1_Row1_Module1_Back
Saved: results\irr_1axis_2021-01-13_1100_Scene1_Row1_Module1.csv
Index: 2021-01-13_1100. Wm2Front: 253.20913333333337. Wm2Back: 52.77725
Linescan in process: 1axis_2021-01-13_1200_Scene1_Row1_Module1_Front
Linescan in process: 1axis_2021-01-13_1200_Scene1_Row1_Module1_Back
Saved: results\irr_1axis_2021-01-13_1200_Scene1_Row1_Module1.csv
Index: 2021-01-13_1200. Wm2Front: 252.7144333

In [12]:
# Include an AgriPV groundscan too
trackerdict = demo.analysis1axisground(sceneNum=1, sensorsground=10)

Linescan in process: 1axis_groundscan_2021-01-13_1100_Row1_Module1_Front
Saved: results\irr_1axis_groundscan_2021-01-13_1100_Row1_Module1.csv
Index: 2021-01-13_1100. Wm2Ground: 204.88420000000002. sensorsground: 10
Linescan in process: 1axis_groundscan_2021-01-13_1200_Row1_Module1_Front
Saved: results\irr_1axis_groundscan_2021-01-13_1200_Row1_Module1.csv
Index: 2021-01-13_1200. Wm2Ground: 237.60913. sensorsground: 10


In [13]:
# show the initial irradiance results before continuing:
demo.results

,timestamp,name,modNum,rowNum,sceneNum,x,y,z,mattype,rearMat,Wm2Front,Wm2Back,backRatio,surf_azm,surf_tilt,theta,temp_air
0,2021-01-13_1100,1axis_2021-01-13_1100_Scene0,3,1,0,"[0.3734448, 0.01462469, -0.3441954]","[2.28669e-17, 8.955042e-19, -2.107589e-17]","[1.166863, 1.51507, 1.863277]","[a2.0.a0.test-module.6457, a2.0.a0.test-module...","[a2.0.a0.test-module.2310, a2.0.a0.test-module...","[254.39559999999997, 255.37389999999996, 253.9...","[39.44632, 39.78179, 40.5239]","[0.15505836162904693, 0.15577799540988566, 0.1...",90.0,44.14,-44.14,4.6
1,2021-01-13_1100,1axis_2021-01-13_1100_Scene1,1,1,1,"[-14.62656, -14.98538, -15.3442]","[2.28669e-17, 8.955042e-19, -2.107589e-17]","[2.166863, 2.51507, 2.863277]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[a0.0.a0.test.2310, a0.0.a0.test.2310, a0.0.a0...","[252.61250000000004, 253.4609, 253.554]","[52.48218, 52.95898, 52.89059]","[0.20775683009815385, 0.20894256691045082, 0.2...",90.0,44.14,-44.14,4.6
2,2021-01-13_1100,1axis_groundscan_2021-01-13_1100,1,1,1,"[-15.0, -14.44444, -13.88889, -13.33333, -12.7...","[0.0, 3.401797e-17, 6.803593e-17, 1.020539e-16...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...","[groundplane, groundplane, groundplane, ground...",NaN,"[199.1851, 201.577, 204.3959, 206.5729, 207.20...",NaN,NaN,90.0,44.14,-44.14,4.6
3,2021-01-13_1200,1axis_2021-01-13_1200_Scene0,3,1,0,"[0.473756, 0.007594116, -0.4585678]","[2.900919e-17, 4.650055e-19, -2.807918e-17]","[1.338767, 1.519579, 1.700391]","[a2.0.a0.test-module.6457, a2.0.a0.test-module...","[a2.0.a0.test-module.2310, a2.0.a0.test-module...","[253.5427, 253.6665, 252.8495]","[37.92271, 37.79234, 38.71431]","[0.14957070516837925, 0.1489837681216553, 0.15...",90.0,21.20,-21.20,6.5
4,2021-01-13_1200,1axis_2021-01-13_1200_Scene1,1,1,1,"[-14.52624, -14.99241, -15.45857]","[2.900919e-17, 4.650055e-19, -2.807918e-17]","[2.338767, 2.519579, 2.700391]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[a0.0.a0.test.2310, a0.0.a0.test.2310, a0.0.a0...","[252.5916, 252.71439999999998, 252.8373]","[50.603010000000005, 50.97375, 50.65398]","[0.20033449119253693, 0.20170416998726634, 0.2...",90.0,21.20,-21.20,6.5
5,2021-01-13_1200,1axis_groundscan_2021-01-13_1200,1,1,1,"[-15.0, -14.44444, -13.88889, -13.33333, -12.7...","[0.0, 3.401797e-17, 6.803593e-17, 1.020539e-16...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...","[groundplane, groundplane, groundplane, ground...",NaN,"[228.1314, 230.3008, 233.4675, 236.7338, 239.4...",NaN,NaN,90.0,21.20,-21.20,6.5


## Calculating the Performance and Exporting the Results to a CSV

In [14]:
# Calculate performance.
compiledResults = demo.calculatePerformance1axis()
print(f'\nCompiled results:\n')
display(compiledResults)


No CECModule data passed; using default for Prism Solar BHC72-400

Compiled results:



,timestamp,name,modNum,rowNum,sceneNum,x,y,z,mattype,rearMat,...,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
0,2021-01-13_1100,1axis_2021-01-13_1100_Scene0,3,1,0,"[0.3734448, 0.01462469, -0.3441954]","[2.28669e-17, 8.955042e-19, -2.107589e-17]","[1.166863, 1.51507, 1.863277]","[a2.0.a0.test-module.6457, a2.0.a0.test-module...","[a2.0.a0.test-module.2310, a2.0.a0.test-module...",...,91.858907,80.410596,14.111711,14.237317,0.021773,91.838906,3.8,144,149,211
1,2021-01-13_1100,1axis_2021-01-13_1100_Scene1,1,1,1,"[-14.62656, -14.98538, -15.3442]","[2.28669e-17, 8.955042e-19, -2.107589e-17]","[2.166863, 2.51507, 2.863277]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[a0.0.a0.test.2310, a0.0.a0.test.2310, a0.0.a0...",...,123.556838,105.929483,16.674675,16.640651,0.008254,123.546640,3.8,144,149,211
2,2021-01-13_1100,1axis_groundscan_2021-01-13_1100,1,1,1,"[-15.0, -14.44444, -13.88889, -13.33333, -12.7...","[0.0, 3.401797e-17, 6.803593e-17, 1.020539e-16...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...","[groundplane, groundplane, groundplane, ground...",NaN,...,85.622995,85.622995,0.000000,0.000000,0.000000,85.622995,3.8,144,149,211
3,2021-01-13_1200,1axis_2021-01-13_1200_Scene0,3,1,0,"[0.473756, 0.007594116, -0.4585678]","[2.900919e-17, 4.650055e-19, -2.807918e-17]","[1.338767, 1.519579, 1.700391]","[a2.0.a0.test-module.6457, a2.0.a0.test-module...","[a2.0.a0.test-module.2310, a2.0.a0.test-module...",...,90.292632,79.426544,13.549799,13.680676,0.018729,90.275721,3.9,97,200,249
4,2021-01-13_1200,1axis_2021-01-13_1200_Scene1,1,1,1,"[-14.52624, -14.99241, -15.45857]","[2.900919e-17, 4.650055e-19, -2.807918e-17]","[2.338767, 2.519579, 2.700391]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[a0.0.a0.test.2310, a0.0.a0.test.2310, a0.0.a0...",...,121.853928,105.009484,16.063532,16.040879,0.006446,121.846073,3.9,97,200,249
5,2021-01-13_1200,1axis_groundscan_2021-01-13_1200,1,1,1,"[-15.0, -14.44444, -13.88889, -13.33333, -12.7...","[0.0, 3.401797e-17, 6.803593e-17, 1.020539e-16...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...","[groundplane, groundplane, groundplane, ground...",NaN,...,98.715680,98.715680,0.000000,0.000000,0.000000,98.715680,3.9,97,200,249


In [15]:
demo.exportTrackerDict(savefile=os.path.join('results','Final_Results.csv'),reindex=False)
pd.read_csv(os.path.join('results','Final_Results.csv'))

Exporting TrackerDict
3
3
1
1


,Unnamed: 0,timestamp,rowNum,modNum,sceneNum,name,Wm2Front,Wm2Back,DNI,DHI,...,theta,surf_tilt,surf_azm,POA_eff,Gfront_mean,Grear_mean,Pout_raw,Mismatch,Pout,Pout_Gfront
0,0,2021-01-13_1100,1,3,0,1axis_2021-01-13_1100_Scene0,[254.3956 255.3739 253.9707],[39.44632 39.78179 40.5239 ],144,149,...,-44.14,44.14,90.0,290.505670,254.580067,39.917337,91.858907,0.021773,91.838906,80.410596
1,1,2021-01-13_1100,1,1,1,1axis_2021-01-13_1100_Scene1,[252.6125 253.4609 253.554 ],[52.48218 52.95898 52.89059],144,149,...,-44.14,44.14,90.0,295.430933,253.209133,52.777250,123.556838,0.008254,123.546640,105.929483
2,2,2021-01-13_1100,1,1,1,1axis_groundscan_2021-01-13_1100,[199.1851 201.577 204.3959 206.5729 207.2027 ...,NaN,144,149,...,-44.14,44.14,90.0,204.884200,204.884200,0.000000,85.622995,0.000000,85.622995,85.622995
3,3,2021-01-13_1200,1,3,0,1axis_2021-01-13_1200_Scene0,[253.5427 253.6665 252.8495],[37.92271 37.79234 38.71431],97,200,...,-21.20,21.20,90.0,287.681708,253.352900,38.143120,90.292632,0.018729,90.275721,79.426544
4,4,2021-01-13_1200,1,1,1,1axis_2021-01-13_1200_Scene1,[252.5916 252.7144 252.8373],[50.60301 50.97375 50.65398],97,200,...,-21.20,21.20,90.0,293.309297,252.714433,50.743580,121.853928,0.006446,121.846073,105.009484
5,5,2021-01-13_1200,1,1,1,1axis_groundscan_2021-01-13_1200,[228.1314 230.3008 233.4675 236.7338 239.4279 ...,NaN,97,200,...,-21.20,21.20,90.0,237.609130,237.609130,0.000000,98.715680,0.000000,98.715680,98.715680


## Now look at gencumulativesky tracking workflow

In [16]:
starttime = '01_13_11';  endtime = '12_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
#metdata = demo.readWeatherFile(weatherFile=weatherfile)
demo.setGround(0.2)
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



In [17]:
sceneDict = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':5, 'nRows': 2}
trackerdict = demo.set1axis(metdata=metdata, cumulativesky=True, limit_angle=15, backtrack=False)
trackerdict = demo.genCumSky1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(modWanted = [2,4], sensorsy=3)
trackerdict = demo.analysis1axisground(sensorsground=10)

Saving file EPWs\1axis_-15.0.csv, # points: 1875
Saving file EPWs\1axis_-10.0.csv, # points: 241
Saving file EPWs\1axis_0.0.csv, # points: 2
Saving file EPWs\1axis_5.0.csv, # points: 242
Saving file EPWs\1axis_10.0.csv, # points: 33
Saving file EPWs\1axis_15.0.csv, # points: 1683
message: There were 1833 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-15.0.rad
message: There were 238 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-10.0.rad
message: There were 2 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_0.0.rad
message: There were 239 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_5.0.rad
message: There were 33 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_10.0.rad
message: There were 1661 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_15.0.rad

Makin

In [18]:
results = demo.calculatePerformance1axis() # saves to demo.compiledResults and results/Cumulative_Results.csv
print('\nCompiled results:\n')
display(demo.compiledResults)


Compiled results:



,rowNum,modNum,sceneNum,BGG,Gfront_mean,Grear_mean,POA_eff,Wm2Front,Wm2Back
0,1,2,0,11.280768,1.666482e+06,208879.957100,"[1856092.1003633335, 1852229.9246933332, 18550...","[1670939.5391, 1670103.7703999998, 1658402.651...","[210677.9037, 206386.5974, 209575.3702]"
1,1,3,0,NaN,1.126698e+06,NaN,"[0.0, 0.0, 0.0]","[628995.037, 704936.1219, 865078.2239000001, 1...","[nan, nan, nan]"
2,1,4,0,10.303140,1.661373e+06,190192.838167,"[1834699.018923333, 1829820.722463333, 1833118...","[1665089.0180000002, 1665383.926, 1653645.085,...","[192584.82510000005, 187164.4957, 190829.1937]"


In [19]:
# Results are also automatically saved in \results\Cumulative_Results.csv
pd.read_csv(os.path.join('results','Cumulative_Results.csv'))

,rowNum,modNum,sceneNum,BGG,Gfront_mean,Grear_mean,POA_eff,Wm2Front,Wm2Back
0,1,2,0,11.281,1666481.987,208879.957,"[1856092.1003633335, 1852229.9246933332, 18550...","[1670939.5391, 1670103.7703999998, 1658402.651...","[210677.9037, 206386.5974, 209575.3702]"
1,1,3,0,NaN,1126697.731,NaN,"[0.0, 0.0, 0.0]","[628995.037, 704936.1219, 865078.2239000001, 1...","[nan, nan, nan]"
2,1,4,0,10.303,1661372.676,190192.838,"[1834699.018923333, 1829820.722463333, 1833118...","[1665089.0180000002, 1665383.926, 1653645.085,...","[192584.82510000005, 187164.4957, 190829.1937]"
